In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("data/output/final_sentence_pool_annotated.parquet")

# Some Analysis

In [3]:
df["final_label"].value_counts()

0    39429
1    26480
Name: final_label, dtype: int64

In [4]:
df[["label_zephyr","label_openchat","label_llama","final_label"]].value_counts(normalize=True)

label_zephyr  label_openchat  label_llama  final_label
0             0               0            0              0.440547
1             1               1            1              0.269341
0             1               0            0              0.103840
1             1               0            1              0.065424
0             1               1            1              0.058019
              0               1            0              0.036717
1             0               0            0              0.017130
                              1            1              0.008982
dtype: float64

In [5]:
df[["source_party","final_label"]].value_counts(normalize=True)

source_party  final_label
Lean Left     0              0.152195
Center        0              0.133988
Left          0              0.131970
Right         1              0.124080
              0              0.096057
Left          1              0.094297
Lean Right    0              0.084025
Lean Left     1              0.074406
Lean Right    1              0.068883
Center        1              0.040101
dtype: float64

# Remove sentence (conservative)

In [6]:
import re

# strange starts
df = df[~df['text'].str.islower()]
df = df[~df['text'].str.startswith("(")]
df = df[~df['text'].str.startswith("*")]
df = df[~df['text'].str.startswith("-")]
df = df[~df['text'].str.startswith("🔥")]
df = df[~df['text'].str.startswith("$")]
df = df[~df['text'].str.startswith("➼")]
df = df[~df['text'].str.startswith("📹")]
df = df[~df['text'].str.startswith("♪")]
df = df[~df['text'].str.startswith("♪")]
df = df[~df['text'].str.startswith("†")]
df = df[~df['text'].str.startswith("♬")]
df = df[~df['text'].str.startswith(">")]
df = df[~df['text'].str.startswith(":")]
pattern = re.compile(r"\d+% \(\d+ Votes\) \d+% \(\d+ Votes\)")
# Use str.contains with the regex pattern to filter rows
df = df[~df['text'].str.contains(pattern)]
df = df[~df['text'].str.startswith(".")]
df = df[~df['text'].str.startswith("!")]
df = df[~df['text'].str.startswith('"')]

# TODO exclude stuff starting with 'YOU' variations <- opinionated
df = df[~df['text'].str.startswith("You ")]
df = df[~df['text'].str.startswith("You’ve ")]
df = df[~df['text'].str.startswith("You’ll ")]
df = df[~df['text'].str.startswith("You’re ")]
df = df[~df['text'].str.startswith("You’d ")]

# TODO exclude stuff starting with 'I' variations <- opinionated
df = df[~df['text'].str.startswith("I've ")]
df = df[~df['text'].str.startswith("I’ve ")]
df = df[~df['text'].str.startswith("I’m ")]
df = df[~df['text'].str.startswith("I'm ")]
df = df[~df['text'].str.startswith("I'd ")]
df = df[~df['text'].str.startswith("I'll ")]
df = df[~df['text'].str.startswith("I,")]

indices_to_check = [6557, 58590, 17440, 43192, 10127, 27994, 35381]
rows_to_drop = df[df.index.isin(indices_to_check)]
df.drop(rows_to_drop.index, inplace=True)

df.to_parquet("data/output/final_sentence_pool_annotated_conservative.parquet", index=False)
df

,text,source_party,source_name,bias_estimate,model_uncertainity,sentence_id,article_id,label_zephyr,response_zephyr,label_openchat,response_openchat,label_llama,response_llama,final_label
0,"Advertisement Initially, the conference attrac...",Right,townhall,0,0,bd51e6ee-8e88-4901-875d-8c5151b42692,f87a85cb-a8d7-4a63-b71d-50342f2022c3,0,The sentence is not biased because it presents...,0,The sentence is not biased because it presents...,0,The sentence is not biased because it presents...,0
1,The number of workers maintaining wind turbine...,Left,alternet,0,0,681738a3-6f56-4124-aae4-52bf803c4d22,62e545ca-11ee-44be-9422-54eff5b1680e,0,The sentence presents a factual statement abou...,0,The sentence provides factual information from...,0,The sentence presents a factual statement abou...,0
2,How did a virus like this get from a bat to a ...,Left,alternet,1,0,1050d536-e626-4647-9cc5-12b731d99077,f54d2a6e-a1dd-42a5-af86-866e5fe8680e,0,The sentence is classified as not biased becau...,0,The sentence is classified as not biased becau...,0,The sentence is not biased because it is a fac...,0
3,"Sin was conceived in a garden, takes root in e...",Lean Right,the-christian-post,1,0,d3910f76-fca5-44cf-98ff-c89b744743e2,12f3585f-d850-4d33-8b1c-d4e25cc4896b,1,The sentence is biased because it presents a r...,1,The sentence is biased because it presents a r...,1,The sentence is biased because it presents a r...,1
4,The two countries have warred over this territ...,Lean Right,the-christian-post,0,0,c9c2c432-d67c-450b-b6d3-1fbd3e4951df,715e6c9e-1b12-4341-9f26-3c00c246ee1b,0,The sentence is not biased because it presents...,0,The sentence is not biased because it presents...,0,The sentence is not biased because it presents...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65904,The Gig Economy is Coming. What Will It Mean F...,Left,alternet,0,0,57e37079-3ab5-4a80-95a7-867256f089e7,8c907329-affd-4446-9ea4-0bdae1ed04cd,0,The sentence is neutral because it presents a ...,0,The sentence is a neutral statement that intro...,0,The sentence presents a question that invites ...,0
65905,"He arrived, freshly graduated, and told me he ...",Center,christian-science-monitor,0,0,4593350c-36d1-44ed-b6b0-4fb8655b55b7,fb4c99c1-488b-4337-be4d-c2208a249602,0,The sentence is not biased because it presents...,0,The sentence is not biased because it presents...,0,The sentence is neutral and not biased because...,0
65906,This is not a good time for American higher ed...,Right,american-thinker,1,0,92adcba8-bd22-4c3e-8bd5-af58c45ce9fa,e517de10-85b8-4273-b392-3495a70f4af5,0,The sentence is classified as not biased becau...,1,The sentence is biased because it uses loaded ...,1,The sentence is biased because it uses negativ...,1
65907,Halls pleaded no contest in March to a misdeme...,Center,cnbc,0,0,81f1f6cf-b8e6-436d-ad2b-3d56c4e9c110,8e89132c-acc2-4b5e-a6e4-b51a9f2d4d2e,0,The sentence presents a factual account of an ...,0,"The sentence is factual and neutral, providing...",0,The sentence reports on a legal matter without...,0


In [7]:
df.final_label.value_counts()

0    38704
1    26008
Name: final_label, dtype: int64

In [8]:
df[["label_zephyr","label_openchat","label_llama","final_label"]].value_counts(normalize=True)

label_zephyr  label_openchat  label_llama  final_label
0             0               0            0              0.440537
1             1               1            1              0.269795
0             1               0            0              0.103659
1             1               0            1              0.065645
0             1               1            1              0.057439
              0               1            0              0.036577
1             0               0            0              0.017323
                              1            1              0.009025
dtype: float64

In [9]:
df[["source_party"]].value_counts()

source_party
Lean Left       14680
Left            14657
Right           14263
Center          11207
Lean Right       9905
dtype: int64

# Create final sample out of this

- we want ~ equal labels
- we want ~ equal parties

In [10]:
def _balanced_sampling(group):
    min_count = group.groupby(["final_label"]).size().min()
    def sample_subgroup(subgroup):
        return subgroup.sample(n=min_count,random_state=42)
    return (
        group.groupby(["final_label"])
        .apply(sample_subgroup)
        .reset_index(drop=True)
    )

In [11]:
final_sample_balanced = df.groupby(["source_party"]).apply(_balanced_sampling).reset_index(drop=True)
print(final_sample_balanced.source_party.value_counts())
print(final_sample_balanced.final_label.value_counts())
print(final_sample_balanced[final_sample_balanced.source_party=='Center'].final_label.value_counts())
print(final_sample_balanced[final_sample_balanced.source_name=='abc-news'].final_label.value_counts())




Right         12420
Left          12202
Lean Left      9634
Lean Right     8902
Center         5172
Name: source_party, dtype: int64
0    24165
1    24165
Name: final_label, dtype: int64
0    2586
1    2586
Name: final_label, dtype: int64
0    297
1    148
Name: final_label, dtype: int64


In [12]:
import numpy as np
print("Stats per outlet")
print("mean:",np.mean(final_sample_balanced.groupby('source_name')['final_label'].mean().tolist()))
print("std:",np.std(final_sample_balanced.groupby('source_name')['final_label'].mean().tolist()))


Stats per outlet
mean: 0.4905878239535543
std: 0.16492191066623096


In [13]:
final_sample_balanced.to_parquet("data/output/anno-lexical.parquet", index=False)